In [18]:
import pandas as pd
import matplotlib.pyplot as plt

In [19]:
uriage = pd.read_csv('./chapter02/uriage.csv')
kokyaku = pd.read_excel('./chapter02/kokyaku_daicho.xlsx')

In [20]:
uriage.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [25]:
uriage['purchase_date_dtype'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_month'] = uriage['purchase_date_dtype'].dt.strftime('%Y%m')
res = uriage.pivot_table(index='purchase_month',columns='item_name',aggfunc='size',fill_value=0)
res

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [6]:
kokyaku.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [24]:
print(len(pd.unique(uriage["item_name"])))

26


In [23]:
uriage["item_name"] = uriage["item_name"].str.upper().str.replace("　","").str.replace(" ","")

In [30]:
uriage.isnull().any(axis=0)

purchase_date          False
item_name              False
item_price             False
customer_name          False
purchase_date_dtype    False
purchase_month         False
dtype: bool

金額がNullになっている行を抜き出す。
重複を除去して、金額Nullの商品名を取り出しループ
金額Null以外で同一商品名のリストを取り出し最大値を取得。
金額Nullの同一商品名の金額を埋める

In [28]:
flg_is_null = uriage["item_price"].isnull()
for trg in list(uriage.loc[flg_is_null,"item_name"].unique()):
    price = uriage.loc[(~flg_is_null)&(uriage["item_name"]== trg),"item_price"].max()
    uriage.loc[(flg_is_null)&(uriage["item_name"]== trg),"item_price"] = price
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_date_dtype,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,2019-06-13 18:02:34,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,2019-07-13 13:05:29,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,2019-05-11 19:42:07,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,2019-02-12 23:40:45,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,2019-04-22 03:09:35,201904


In [35]:
kokyaku["登録日"].head(20)

0     2018/01/04
1          42782
2     2018/01/07
3          42872
4          43127
5     2017/06/20
6     2018/06/11
7     2017/05/19
8     2018/02/12
9     2017/07/05
10    2018/03/31
11    2017/04/22
12    2018/03/09
13    2017/03/13
14    2018/01/24
15    2017/06/09
16    2018/05/02
17    2017/02/05
18    2018/07/10
19    2017/07/08
Name: 登録日, dtype: object

In [32]:
uriage["customer_name"].head()

0    深井菜々美
1     浅田賢二
2     南部慶二
3     麻生莉緒
4     平田鉄二
Name: customer_name, dtype: object

In [33]:
kokyaku["顧客名"] = kokyaku["顧客名"].str.replace("　","").str.replace(" ","")

In [50]:
flg_is_digit = kokyaku["登録日"].astype("str").str.isdigit()

In [39]:
fromSerial = pd.to_timedelta(kokyaku.loc[flg_is_digit,"登録日"].astype("float") -2,unit ="D") + pd.to_datetime('1900/1/1')
fromSerial

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 登録日, dtype: datetime64[ns]

In [40]:
fromString = pd.to_datetime(kokyaku.loc[~flg_is_digit,"登録日"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [41]:
kokyaku["登録日"] = pd.concat([fromSerial,fromString])
kokyaku

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-16
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-17
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-27
...,...,...,...,...,...
195,川上りえ,かわかみ りえ,G市,kawakami_rie@example.com,2017-06-20
196,小松季衣,こまつ としえ,E市,komatsu_toshie@example.com,2018-06-20
197,白鳥りえ,しらとり りえ,F市,shiratori_rie@example.com,2017-04-29
198,大西隆之介,おおにし りゅうのすけ,H市,oonishi_ryuunosuke@example.com,2019-04-19


In [49]:
kokyaku["登録年月"] = kokyaku["登録日"].dt.strftime("%Y%m")
rslt = kokyaku.groupby("登録年月").count()
print(rslt["顧客名"])

登録年月
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 顧客名, dtype: int64


In [51]:
join_data = pd.merge(uriage,kokyaku,left_on="customer_name",right_on="顧客名",how="left")
join_data = join_data.drop("customer_name",axis = 1)

In [52]:
join_data

,purchase_date,item_name,item_price,purchase_date_dtype,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,2019-06-13 18:02:34,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,2019-07-13 13:05:29,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,2019-05-11 19:42:07,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,2019-02-12 23:40:45,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,2019-04-22 03:09:35,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706
...,...,...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,商品Y,2500.0,2019-02-15 02:56:39,201902,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01,201707
2995,2019-06-22 04:03:43,商品M,1300.0,2019-06-22 04:03:43,201906,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31,201803
2996,2019-03-29 11:14:05,商品Q,1700.0,2019-03-29 11:14:05,201903,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15,201703
2997,2019-07-14 12:56:49,商品H,800.0,2019-07-14 12:56:49,201907,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13,201807


In [53]:
dump_data = join_data[["purchase_date","purchase_month","item_name","item_price","顧客名","かな","地域","メールアドレス","登録日"]]
dump_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13


In [54]:
dump_data.to_csv("dump_data.csv",index=False)

In [56]:
import_data = pd.read_csv("./dump_data.csv")
import_data

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:39,201902,商品Y,2500.0,福島友也,ふくしま ともや,B市,fukushima_tomoya@example.com,2017-07-01
2995,2019-06-22 04:03:43,201906,商品M,1300.0,大倉晃司,おおくら こうじ,E市,ookura_kouji@example.com,2018-03-31
2996,2019-03-29 11:14:05,201903,商品Q,1700.0,尾形小雁,おがた こがん,B市,ogata_kogan@example.com,2017-03-15
2997,2019-07-14 12:56:49,201907,商品H,800.0,芦田博之,あしだ ひろゆき,E市,ashida_hiroyuki@example.com,2018-07-13


In [ ]:
byprice = import_data.pivot_table(index="purchase_month",columns="item_name",aggfunc="size",fill_value=0)